In [19]:
import pandas as pd
import numpy as np 

iterations = 2000
n_runs = 10
L = 50
rho_values = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
runs = list(range(10))
results = []
dfs = {}

for rho_index, rho in enumerate(rho_values):
    df = pd.DataFrame(columns = [f'run_{i}' for i in runs])
    for run in runs:
        my_data = np.genfromtxt(f'./results/run_{run}_rho_{rho}_L_{L}_iters_{iterations}.csv', delimiter = ',', skip_header=1)
        my_data = my_data.T
        df[f'run_{run}'] = my_data[1]
    df['mean'] = df.mean(axis = 1)
    df['std'] = df.std(axis=1)
    dfs[rho_index] = df

final_df = pd.concat(dfs.values(), axis=1)

final_df
        

        
    

,run_0,run_1,run_2,run_3,run_4,run_5,run_6,run_7,run_8,run_9,...,run_2,run_3,run_4,run_5,run_6,run_7,run_8,run_9,mean,std
0,0.469496,0.46688,0.446137,0.454669,0.611046,0.431010,0.479167,0.425452,0.227600,0.472991,...,0.007306,0.014665,0.011440,0.007601,0.006508,0.003597,0.026694,0.000867,0.008238,0.007626
1,0.469496,0.06704,0.169205,0.454669,0.611046,0.146515,0.088542,1.000000,0.227600,-0.185771,...,-0.001100,0.031582,0.022630,0.022194,0.031403,0.016576,0.033546,0.024262,0.019902,0.011164
2,0.602122,0.60016,0.169205,0.182004,0.611046,0.146515,0.479167,1.000000,-0.158599,0.209486,...,0.012173,0.020280,0.024415,0.029623,0.038848,0.020282,0.039505,0.005827,0.023074,0.012418
3,0.204244,0.46688,0.169205,0.454669,0.611046,0.146515,0.479167,1.000000,0.098867,0.472991,...,0.021727,0.034576,-0.008640,0.005680,0.027818,0.026466,0.056267,0.014825,0.024987,0.018788
4,0.204244,0.06704,0.169205,0.045671,0.611046,0.573257,0.479167,0.425452,-0.287333,0.604743,...,0.014100,0.031794,0.003900,-0.004472,0.036310,0.030120,0.048496,0.019883,0.026313,0.019203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.602122,-0.19952,-0.453891,-0.022495,1.000000,0.573257,-0.106771,0.138179,-0.158599,-0.317523,...,0.054620,-0.003016,0.031567,-0.034311,0.031715,-0.036429,-0.055862,-0.051428,-0.016251,0.039449
1996,0.602122,0.20032,-0.384658,0.045671,1.000000,1.000000,-0.497396,0.138179,0.098867,-0.581028,...,0.054620,-0.003016,0.031567,-0.034311,0.027132,-0.036429,-0.055862,-0.051428,-0.016709,0.038912
1997,0.204244,0.20032,-0.800055,0.045671,1.000000,1.000000,-0.106771,-0.292732,0.227600,-0.844532,...,0.054620,-0.003016,0.031567,-0.034311,0.028309,-0.036429,-0.055862,-0.051428,-0.016591,0.039046
1998,-0.193634,0.60016,-0.661590,0.045671,1.000000,1.000000,0.414062,-0.723643,0.613800,-0.317523,...,0.054620,-0.003016,0.031567,-0.034311,0.027661,-0.036429,-0.055862,-0.051428,-0.016656,0.038972


In [6]:
import numpy as np
import scipy.stats as stats

run = 0
rho = 0.1
L=50
iterations = 20000
phi_values = np.genfromtxt(f'./results/run_{run}_rho_{rho}_L_{L}_iters_{iterations}.csv', delimiter = ',', skip_header=1)
phi_values = phi_values[:,1]
for i in range(iterations):
    if i > 5200:
        slope, intercept, r, p_values, se = stats.linregress([np.linspace(0, 4999, 5000), phi_values[i-5000:i]])
        if p_values > 0.05:
            print(i, p_values)

8446 0.05293167473747882
8447 0.05997372734584152
8448 0.07000922507446483
8449 0.07963825707655178
8450 0.0902026361533904
8451 0.10036163201937674
8452 0.11709791397879499
8453 0.132999875708948
8454 0.14694017148982408
8455 0.16308103735564616
8456 0.17780508701095632
8457 0.1961730342204347
8458 0.21773703117479673
8459 0.2434021837736963
8460 0.26563337514588503
8461 0.2874990939684456
8462 0.30856091913100503
8463 0.3244797432473462
8464 0.3390738805568504
8465 0.3577923104021512
8466 0.3856798742442239
8467 0.4159617281432647
8468 0.45271549597242533
8469 0.4880274747870751
8470 0.5128822914148776
8471 0.5407948090092733
8472 0.5551536150467097
8473 0.5653277111883042
8474 0.5819248891454047
8475 0.6053849252510389
8476 0.6379869048285882
8477 0.6670085113072403
8478 0.6884594354923437
8479 0.7220545131708496
8480 0.7629624161821871
8481 0.8087345289549136
8482 0.8538640873683703
8483 0.8823789572610842
8484 0.9112214577877554
8485 0.925941713263926
8486 0.9360088542419723
8487 